In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

class HTMLTableParser:

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [self.parse_html_table(table) for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            column_names = column_names + [""]*(n_columns - len(column_names))

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df

In [2]:
hp = HTMLTableParser()

url = "http://www.worldfootball.net/schedule/eng-premier-league-2006-2007-spieltag/1"

all_tables = hp.parse_url(url)

all_tables[3].columns = ['#', '', 'Team', 'M.', 'W', 'D', 'L', 'goals', 'Dif.', 'Pt.']
all_tables[3]

,#,,Team,M.,W,D,L,goals,Dif.,Pt.
0,1,\n\n,\nManchester United\n,1.0,1.0,0.0,0.0,5:1,4.0,3.0
1,2,\n\n,\nChelsea FC\n,1.0,1.0,0.0,0.0,3:0,3.0,3.0
2,,\n\n,\nPortsmouth FC\n,1.0,1.0,0.0,0.0,3:0,3.0,3.0
3,4,\n\n,\nWest Ham United\n,1.0,1.0,0.0,0.0,3:1,2.0,3.0
4,5,\n\n,\nBolton Wanderers\n,1.0,1.0,0.0,0.0,2:0,2.0,3.0
5,6,\n\n,\nReading FC\n,1.0,1.0,0.0,0.0,3:2,1.0,3.0
6,7,\n\n,\nEverton FC\n,1.0,1.0,0.0,0.0,2:1,1.0,3.0
7,,\n\n,\nNewcastle United\n,1.0,1.0,0.0,0.0,2:1,1.0,3.0
8,9,\n\n,\nArsenal FC\n,1.0,0.0,1.0,0.0,1:1,0.0,1.0
9,,\n\n,\nAston Villa\n,1.0,0.0,1.0,0.0,1:1,0.0,1.0


In [3]:
seasons = ['2008/2009', '2009/2010', '2010/2011', '2011/2012', '2012/2013', '2013/2014', '2014/2015', '2015/2016']
stages  = range(1, 39)
fetures = ["Pts", "Norm_Pts", "Position","Goals_Diff","Pos_Goals","Neg_Goals"]
teams_per_season = 20

In [4]:
table_dic = {}

for season in seasons:
    for stage in stages:
        url = "http://www.worldfootball.net/schedule/eng-premier-league-" + season.replace("/","-") + "-spieltag/" + str(stage)
        table = hp.parse_url(url)[3]
        table.columns = ['#', '', 'Team', 'M.', 'W', 'D', 'L', 'goals', 'Dif.', 'Pt.']
        table = table[["Team", "Pt.", "Dif.", "goals"]]
        table["Team"] = table["Team"].apply(lambda x: x.split("\n")[1])
        table.sort_values("Team", ascending = True, inplace = True)
        table.reset_index(inplace = True)
        table = table[['Team', 'Pt.', "Dif.", "goals", 'index']]
        table.columns = ['Team', 'Pt.', "Dif.", "goals", 'position']
        table["position"] = table["position"] + 1
        table_dic[(season,stage)] = table

In [5]:
all_teams   = []
all_seasons = seasons * teams_per_season
all_stages  = (list(stages)+[0]) * len(fetures)
all_fetures = fetures * len(all_stages)
for season in seasons:
    all_teams = all_teams + list(table_dic[(season,1)]["Team"].unique())
    
all_seasons.sort()
all_stages.sort()

new_teams = []
for team in all_teams:
    new_teams.append(team.split(" FC")[0].split(" AFC")[0].split("AFC ")[-1])
    
all_teams = new_teams

In [6]:
index_arrays   = [all_seasons, all_teams]
columns_arrays = [all_stages, all_fetures]

index_tuples   = list(zip(*index_arrays))
columns_tuples = list(zip(*columns_arrays))

index   = pd.MultiIndex.from_tuples(index_tuples, names = ['Season', 'Team'])
columns = pd.MultiIndex.from_tuples(columns_tuples, names = ['Stage', ''])

In [7]:
table_per_stage = pd.DataFrame(np.zeros((len(all_teams),len(all_stages))), index = index, columns = columns)

In [8]:
for season in seasons:
    for stage in stages:
        table_per_stage[stage]["Pts"][season]        = table_dic[(season,stage)]["Pt."]
        table_per_stage[stage]["Norm_Pts"][season]   = table_dic[(season,stage)]["Pt."]/table_dic[(season,stage)]["Pt."].max()
        table_per_stage[stage]["Position"][season]   = table_dic[(season,stage)]["position"]
        table_per_stage[stage]["Goals_Diff"][season] = table_dic[(season,stage)]["Dif."]
        table_per_stage[stage]["Pos_Goals"][season]  = table_dic[(season,stage)]["goals"].apply(lambda x: x.split(":")[0])
        table_per_stage[stage]["Neg_Goals"][season]  = table_dic[(season,stage)]["goals"].apply(lambda x: x.split(":")[1])

C:\Users\asafa\Anaconda3\envs\py35_gpu\lib\site-packages\IPython\core\interactiveshell.py:2881: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
table_per_stage

Stage                            0                                          \
                                Pts Norm_Pts Position Goals_Diff Pos_Goals   
Season    Team                                                               
2008/2009 Arsenal               0.0      0.0      0.0        0.0       0.0   
          Aston Villa           0.0      0.0      0.0        0.0       0.0   
          Blackburn Rovers      0.0      0.0      0.0        0.0       0.0   
          Bolton Wanderers      0.0      0.0      0.0        0.0       0.0   
          Chelsea               0.0      0.0      0.0        0.0       0.0   
          Everton               0.0      0.0      0.0        0.0       0.0   
          Fulham                0.0      0.0      0.0        0.0       0.0   
          Hull City             0.0      0.0      0.0        0.0       0.0   
          Liverpool             0.0      0.0      0.0        0.0       0.0   
          Manchester City       0.0      0.0      0.0        0.0       0.0   
          Manchester United     0.0      0.0      0.0        0.0       0.0   
          Middlesbrough         0.0      0.0      0.0        0.0       0.0   
          Newcastle United      0.0      0.0      0.0        0.0       0.0   
          Portsmouth            0.0      0.0      0.0        0.0       0.0   
          Stoke City            0.0      0.0      0.0        0.0       0.0   
          Sunderland            0.0      0.0      0.0        0.0       0.0   
          Tottenham Hotspur     0.0      0.0      0.0        0.0       0.0   
          West Bromwich Albion  0.0      0.0      0.0        0.0       0.0   
          West Ham United       0.0      0.0      0.0        0.0       0.0   
          Wigan Athletic        0.0      0.0      0.0        0.0       0.0   
2009/2010 Arsenal               0.0      0.0      0.0        0.0       0.0   
          Aston Villa           0.0      0.0      0.0        0.0       0.0   
          Birmingham City       0.0      0.0      0.0        0.0       0.0   
          Blackburn Rovers      0.0      0.0      0.0        0.0       0.0   
          Bolton Wanderers      0.0      0.0      0.0        0.0       0.0   
          Burnley               0.0      0.0      0.0        0.0       0.0   
          Chelsea               0.0      0.0      0.0        0.0       0.0   
          Everton               0.0      0.0      0.0        0.0       0.0   
          Fulham                0.0      0.0      0.0        0.0       0.0   
          Hull City             0.0      0.0      0.0        0.0       0.0   
...                             ...      ...      ...        ...       ...   
2014/2015 Manchester United     0.0      0.0      0.0        0.0       0.0   
          Newcastle United      0.0      0.0      0.0        0.0       0.0   
          Queens Park Rangers   0.0      0.0      0.0        0.0       0.0   
          Southampton           0.0      0.0      0.0        0.0       0.0   
          Stoke City            0.0      0.0      0.0        0.0       0.0   
          Sunderland            0.0      0.0      0.0        0.0       0.0   
          Swansea City          0.0      0.0      0.0        0.0       0.0   
          Tottenham Hotspur     0.0      0.0      0.0        0.0       0.0   
          West Bromwich Albion  0.0      0.0      0.0        0.0       0.0   
          West Ham United       0.0      0.0      0.0        0.0       0.0   
2015/2016 Bournemouth           0.0      0.0      0.0        0.0       0.0   
          Arsenal               0.0      0.0      0.0        0.0       0.0   
          Aston Villa           0.0      0.0      0.0        0.0       0.0   
          Chelsea               0.0      0.0      0.0        0.0       0.0   
          Crystal Palace        0.0      0.0      0.0        0.0       0.0   
          Everton               0.0      0.0      0.0        0.0       0.0   
          Leicester City        0.0      0.0      0.0        0.0       0.0   
          Liverpool   

In [10]:
table_per_stage.to_csv("databases/table_per_stage.csv")